In [5]:
import re
import json
import ast
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
from sklearn.metrics.pairwise import cosine_similarity
from yellowbrick.text import TSNEVisualizer
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.stem import PorterStemmer, WordNetLemmatizer, SnowballStemmer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from gensim.utils import simple_preprocess
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.decomposition import PCA
from sklearn.preprocessing import label_binarize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [6]:
restaurants = pd.read_csv('Res_dataset.csv', encoding='latin1')

In [7]:
# Cleaned Restaurant name by removing unwanted commas and characters

import re

def clean_restaurant_name(name):
    if isinstance(name, str):
        # Removing non-alphabetic characters and extra spaces
        cleaned_name = re.sub(r'[^a-zA-Z\s]', '', name)
        cleaned_name = re.sub(r'\s+', ' ', cleaned_name)
        # Converting to lowercase
        cleaned_name = cleaned_name.lower()
        return cleaned_name.strip()  # Removing leading/trailing spaces
    else:
        return ''

restaurants['Cleaned_Restaurant_Name'] = restaurants['Restaurant_Name'].apply(clean_restaurant_name)

# Printing cleaned restaurant names
for name in restaurants['Cleaned_Restaurant_Name']:
    print(name)

jalsa
spice elephant
royal rasoi
mughal palace
taj tandoor
delhi darbar
coastal spice
royal rajputana
ganga sagar
masala magic
flavors of punjab
darjeeling delights
spice street
spicy sizzlers
malabar spice
spice valley
chaat chatore
paradise platter
tandoori twist
biryani bazaar
coastal cuisine
mughlai magic
punjabi tadka
south indian spice
rajasthani rang
spice route
mughal delights
coastal cravings
spice n rice
nawabi nook
deccan delicacies
tandoori tales
biryani boulevard
masala magic
the tandoor
coastal delights
punjab palace
mysore spice
flavors of kerala
royal rajasthani
hyderabad house
the spice garden
goan delights
spice saga
mughlai mansion
coastal cove
mughal spice
pice temple
paradise bakes
coastal cove
flavors of kerala
rajasthani retreat
punjabi palace
south indian spice
awadhi affair
rajasthani rasoi
coastal crab
bengali bites
marathi masala
goan delights
spice route
coastal catch
lucknowi flavors
andhra spice
maharashtrian morsels
punjabi dhaba
coastal canteen
gujarati 

In [8]:
# Cleaned restaurant descrtption to remove commas and unwanted characters

import re

def clean_description(text):
    if isinstance(text, str):
      # Removal of characters and extra spaces using regular expressions
        cleaned_text = re.sub(r'[^a-zA-Z\s]', '', text)
        cleaned_text = re.sub(r'\s+', ' ', cleaned_text)
        cleaned_text = cleaned_text.lower()
        return cleaned_text
    else:
        return ''

#Cleaning specifice columns
restaurants['Restaurant_Name'] = restaurants['Restaurant_Name'].apply(clean_description)
restaurants['reviews'] = restaurants['reviews'].apply(clean_description)

In [9]:
print(restaurants.columns)

Index(['Restaurant_Name', 'Restaurant_description', 'Location',
       'Restaurant_type', 'recommended_dishes', 'famous_Cuisines',
       'Cost_per_person', 'allergy_food_product', 'menu_without_allergy',
       'menu_with_allergy', 'reviews', 'Cleaned_Restaurant_Name'],
      dtype='object')


In [10]:
# Cleaning location to remove comma between city and country

def clean_location(location):
    # Striping leading and trailing spaces
    cleaned_location = location.strip()
    # Converting to lowercase
    cleaned_location = cleaned_location.lower()
    # Replacing commas with empty string
    cleaned_location = cleaned_location.replace(',', '')
    return cleaned_location

# assigning the 'Location' column to new column "clean_Location"
restaurants['Cleaned_Location'] = restaurants['Location'].apply(clean_location)

# Iterating over the cleaned locations and print
for location in restaurants['Cleaned_Location']:
    print(location)

india banashankari
chicago america
india delhi
india jaipur
india mumbai
india delhi
india kochi
india udaipur
india varanasi
india mumbai
india amritsar
india darjeeling
india hyderabad
india bangalore
india kochi
india jaipur
india delhi
india lucknow
india mumbai
india hyderabad
india chennai
india lucknow
india amritsar
india chennai
india jaipur
india kochi
india delhi
india mumbai
india bangalore
india lucknow
india hyderabad
india delhi
india hyderabad
india mumbai
india delhi
india chennai
india amritsar
india mysore
india kochi
india jaipur
india hyderabad
india bangalore
india goa
india chennai
india lucknow
india mangalore
india agra
india bengaluru
india mumbai
india mangalore
india kochi
india jaipur
india amritsar
india chennai
india lucknow
india udaipur
india udupi
india kolkata
india mumbai
india panaji
india bengaluru
india kochi
india lucknow
india hyderabad
india pune
india amritsar
india kochi
india ahmedabad
india srinagar
india jaipur
india chikmagalur
india vija

In [11]:
###############################correction
def clean_restaurant_type(type_str):

    cleaned_types = [t.strip() for t in type_str.split(',')]
    cleaned_type_str = ', '.join(cleaned_types)
    return cleaned_type_str


restaurants['Cleaned_Restaurant_Type'] = restaurants['Restaurant_type'].apply(clean_restaurant_type)


print(restaurants[['Restaurant_type', 'Cleaned_Restaurant_Type']].head())

                                     Restaurant_type  \
0       Fine Dining, Family Gatherings, Celebrations   
1  Fine Dining, Family Gatherings, Celebrations,T...   
2       Fine Dining, Family Gatherings, Celebrations   
3        Fine Dining, Celebrations, Romantic Dinners   
4  Casual Dining, Family Gatherings, Business Mee...   

                             Cleaned_Restaurant_Type  
0       Fine Dining, Family Gatherings, Celebrations  
1  Fine Dining, Family Gatherings, Celebrations, ...  
2       Fine Dining, Family Gatherings, Celebrations  
3        Fine Dining, Celebrations, Romantic Dinners  
4  Casual Dining, Family Gatherings, Business Mee...  


In [12]:
#############################correction

def clean_recommended_dishes(dishes_str):

    cleaned_dishes = [dish.strip() for dish in dishes_str.split(',')]
    cleaned_dishes_str = ', '.join(cleaned_dishes)
    return cleaned_dishes_str


restaurants['Cleaned_Recommended_Dishes'] = restaurants['recommended_dishes'].apply(clean_recommended_dishes)


print(restaurants[['recommended_dishes', 'Cleaned_Recommended_Dishes']].head())

                                  recommended_dishes  \
0  Pasta, Lunch Buffet, Masala Papad, Paneer Laja...   
1  Churros, Cannelloni, Minestrone Soup, Hot Choc...   
2  Butter Chicken, Rogan Josh, Biryani, Tandoori ...   
3  Laal Maas, Dal Makhani, Murgh Musallam, Ghewar...   
4  Tandoori Chicken, Butter Naan, Chicken Tikka M...   

                          Cleaned_Recommended_Dishes  
0  Pasta, Lunch Buffet, Masala Papad, Paneer Laja...  
1  Churros, Cannelloni, Minestrone Soup, Hot Choc...  
2  Butter Chicken, Rogan Josh, Biryani, Tandoori ...  
3  Laal Maas, Dal Makhani, Murgh Musallam, Ghewar...  
4  Tandoori Chicken, Butter Naan, Chicken Tikka M...  


In [13]:
def clean_famous_cuisines(cuisines_str):
    cleaned_cuisines = [cuisine.strip() for cuisine in cuisines_str.split(',')]
    cleaned_cuisines_str = ', '.join(cleaned_cuisines)
    return cleaned_cuisines_str


restaurants['Cleaned_Famous_Cuisines'] = restaurants['famous_Cuisines'].apply(clean_famous_cuisines)


print(restaurants[['famous_Cuisines', 'Cleaned_Famous_Cuisines']].head())

                                     famous_Cuisines  \
0                     North Indian, Mughlai, Chinese   
1  Chinese, North Indian, Thai,American Spicy Spe...   
2                              North Indian, Mughlai   
3                                Mughlai, Rajasthani   
4                                       North Indian   

                             Cleaned_Famous_Cuisines  
0                     North Indian, Mughlai, Chinese  
1  Chinese, North Indian, Thai, American Spicy Sp...  
2                              North Indian, Mughlai  
3                                Mughlai, Rajasthani  
4                                       North Indian  


In [14]:
# Function to clean the allergy_food_product column
def clean_allergy_food_product(allergy_str):
    # Split the string by commas and strip leading/trailing spaces from each substring
    cleaned_str = [item.strip() for item in allergy_str.split(',')]
    return cleaned_str

# Apply the function to the allergy_food_product column
restaurants['Cleaned_Allergy_Food_Product'] = restaurants['allergy_food_product'].apply(clean_allergy_food_product)

print(restaurants[['allergy_food_product', 'Cleaned_Allergy_Food_Product']].head())

  allergy_food_product Cleaned_Allergy_Food_Product
0               Cheese                     [Cheese]
1                Dairy                      [Dairy]
2                 Nuts                       [Nuts]
3               Gluten                     [Gluten]
4            Shellfish                  [Shellfish]


In [15]:
def clean_menu_without_allergy(text):
    items = text.split(',')
    cleaned_items = [item.strip() for item in items]
    cleaned_text = ', '.join(cleaned_items)
    return cleaned_text

restaurants['Cleaned_Menu_Without_Allergy'] = restaurants['menu_without_allergy'].apply(clean_menu_without_allergy)


print(restaurants[['menu_without_allergy', 'Cleaned_Menu_Without_Allergy']].head())

                                menu_without_allergy  \
0   Lunch Buffet, Masala Papad,Dum Biryani, Sweet...   
1  Churros, Minestrone Soup, Pink Sauce Pasta, Salsa   
2  Butter Chicken, Rogan Josh, Biryani, Tandoori ...   
3      Laal Maas, Dal Makhani, Murgh Musallam, Rabri   
4  Butter Naan, Chicken Tikka Masala, Vegetable B...   

                        Cleaned_Menu_Without_Allergy  
0  Lunch Buffet, Masala Papad, Dum Biryani, Sweet...  
1  Churros, Minestrone Soup, Pink Sauce Pasta, Salsa  
2  Butter Chicken, Rogan Josh, Biryani, Tandoori ...  
3      Laal Maas, Dal Makhani, Murgh Musallam, Rabri  
4  Butter Naan, Chicken Tikka Masala, Vegetable B...  


In [16]:
def clean_menu_with_allergy(text):
    # Split the text based on commas
    items = text.split(',')
    # Remove leading and trailing whitespace from each item
    cleaned_items = [item.strip() for item in items]
    # Join the cleaned items back together with commas
    cleaned_text = ', '.join(cleaned_items)
    return cleaned_text


restaurants['Cleaned_Menu_With_Allergy'] = restaurants['menu_with_allergy'].apply(clean_menu_with_allergy)
print(restaurants[['menu_with_allergy', 'Cleaned_Menu_With_Allergy']].head())

                                   menu_with_allergy  \
0  Pasta, Lunch Buffet, Masala Papad, Paneer Laja...   
1  Churros, Cannelloni, Minestrone Soup, Hot Choc...   
2  Butter Chicken, Rogan Josh, Biryani, Tandoori ...   
3  Laal Maas, Dal Makhani, Murgh Musallam, Ghewar...   
4  Tandoori Chicken, Butter Naan, Chicken Tikka M...   

                           Cleaned_Menu_With_Allergy  
0  Pasta, Lunch Buffet, Masala Papad, Paneer Laja...  
1  Churros, Cannelloni, Minestrone Soup, Hot Choc...  
2  Butter Chicken, Rogan Josh, Biryani, Tandoori ...  
3  Laal Maas, Dal Makhani, Murgh Musallam, Ghewar...  
4  Tandoori Chicken, Butter Naan, Chicken Tikka M...  


In [17]:
def clean_reviews(text):
    if pd.isnull(text):
        return ""

    cleaned_text = ' '.join(text.split())
    cleaned_text = cleaned_text.lower()
    return cleaned_text


restaurants['reviews'] = restaurants['reviews'].fillna('')
restaurants['Cleaned_Reviews'] = restaurants['reviews'].apply(clean_reviews)
print(restaurants[['reviews', 'Cleaned_Reviews']].head())

                                             reviews  \
0  beautiful mughalthemed restaurant the place ha...   
1  spice elephant offers a fiery culinary experie...   
2  exquisite dining experience royal rasoi offers...   
3  majestic dining experience mughal palace truly...   
4  culinary excellence taj tandoor offers an exce...   

                                     Cleaned_Reviews  
0  beautiful mughalthemed restaurant the place ha...  
1  spice elephant offers a fiery culinary experie...  
2  exquisite dining experience royal rasoi offers...  
3  majestic dining experience mughal palace truly...  
4  culinary excellence taj tandoor offers an exce...  


In [18]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [19]:
print(restaurants.columns)

Index(['Restaurant_Name', 'Restaurant_description', 'Location',
       'Restaurant_type', 'recommended_dishes', 'famous_Cuisines',
       'Cost_per_person', 'allergy_food_product', 'menu_without_allergy',
       'menu_with_allergy', 'reviews', 'Cleaned_Restaurant_Name',
       'Cleaned_Location', 'Cleaned_Restaurant_Type',
       'Cleaned_Recommended_Dishes', 'Cleaned_Famous_Cuisines',
       'Cleaned_Allergy_Food_Product', 'Cleaned_Menu_Without_Allergy',
       'Cleaned_Menu_With_Allergy', 'Cleaned_Reviews'],
      dtype='object')


In [20]:
#Added all cleaned columns to an array for further use in functions to recommend restaurants

cleaned_column_types = restaurants[['Cleaned_Restaurant_Type',
       'Cleaned_Recommended_Dishes', 'Cleaned_Famous_Cuisines',
       'Cleaned_Allergy_Food_Product', 'Cleaned_Menu_Without_Allergy',
       'Cleaned_Menu_With_Allergy', 'Cleaned_Reviews', 'Cleaned_Location',
       'Cleaned_Restaurant_Name']].dtypes

print(cleaned_column_types)

Cleaned_Restaurant_Type         object
Cleaned_Recommended_Dishes      object
Cleaned_Famous_Cuisines         object
Cleaned_Allergy_Food_Product    object
Cleaned_Menu_Without_Allergy    object
Cleaned_Menu_With_Allergy       object
Cleaned_Reviews                 object
Cleaned_Location                object
Cleaned_Restaurant_Name         object
dtype: object


In [21]:
#Location must be required
#location ,restaurant type and famouse cuisines

def longfamouscusinebased(Location, sentence, cuisine_sentence):
    restaurants['Cleaned_Location'] = restaurants['Cleaned_Location'].str.lower()
    restaurants['Cleaned_Restaurant_Type'] = restaurants['Cleaned_Restaurant_Type'].str.lower()
    restaurants['Cleaned_Famous_Cuisines'] = restaurants['Cleaned_Famous_Cuisines'].str.lower()

    sentence = sentence.lower()
    cuisine_sentence = cuisine_sentence.lower()
    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()
    sentence_tokens = word_tokenize(sentence)
    cuisine_tokens = word_tokenize(cuisine_sentence)
    sentence_tokens = [lemm.lemmatize(word) for word in sentence_tokens if word.isalnum() and word not in sw]
    cuisine_tokens = [lemm.lemmatize(word) for word in cuisine_tokens if word.isalnum() and word not in sw]

    reqbased = restaurants[restaurants['Cleaned_Location'] == Location.lower()].copy()
    similarities = []
    for index, row in reqbased.iterrows():
        # Tokenize the restaurant type and famous cuisines
        temp_tokens = word_tokenize(row['Cleaned_Restaurant_Type'])
        temp_tokens = [lemm.lemmatize(word) for word in temp_tokens if word.isalnum() and word not in sw]
        cuisine_tokens_copy = cuisine_tokens.copy()

        # Check for common tokens in restaurant type
        rvector = set(temp_tokens).intersection(set(sentence_tokens))

        # Check for common tokens in famous cuisines
        for cuisine_token in cuisine_tokens_copy:
            if cuisine_token in row['Cleaned_Famous_Cuisines']:
                rvector.add(cuisine_token)

        similarities.append(len(rvector))

    reqbased['similarity'] = similarities
    reqbased = reqbased.sort_values(by='similarity', ascending=False)

    return reqbased[['reviews', 'Restaurant_Name', 'Restaurant_description','Restaurant_type','recommended_dishes','menu_without_allergy',
       'menu_with_allergy','Cost_per_person', 'allergy_food_product','similarity']].head(10)

# Define the parameters
location = 'New York City East Village Manhattan'
sentence = 'i want to eat from a Casual Dining restaurant'
cuisine_sentence = 'i want to eat from american restaurant'

# Call the function and store the result
result_df = longfamouscusinebased(location, sentence, cuisine_sentence)

# Print the parameters passed to the function
for index, row in result_df.iterrows():
    print("\nRestaurant Name:", row['Restaurant_Name'])
    print("Restaurant Description:", row['Restaurant_description'])
    print("Restaurant Type:", row['Restaurant_type'])
    print("Recommended Dishes:", row['recommended_dishes'])
    print("Menu with allergies:", row['menu_with_allergy'])
    print("Menu without allergies:", row['menu_without_allergy'])
    print("Allergy food product:", row['allergy_food_product'])
    print("Cost per person:", row['Cost_per_person'])
    print("Reviews:", row['reviews'])
    print("Similarity", row['similarity'])
    print("---------------------------------------------")


Restaurant Name: klong
Restaurant Description: Klong offers a modern interpretation of Thai cuisine, showcasing bold flavors and innovative dishes. From classic staples like pad Thai and green curry to creative offerings such as crispy duck salad and short rib curry, each dish at Klong is crafted with fresh, seasonal ingredients. Expect a harmonious blend of traditional Thai flavors with contemporary twists, resulting in a dining experience that is both exciting and satisfying.
Restaurant Type:  Casual Dining
Recommended Dishes: Pad Thai,Green Curry,Mango Sticky Rice,Tom Yum Soup,Massaman Curry,Greek Gyro Wrap
Menu with allergies: Greek Gyro Wrap
Menu without allergies: Pad Thai,Green Curry,Mango Sticky Rice,Tom Yum Soup,Massaman Curry
Allergy food product: Dairy
Cost per person: $29.05 
Reviews: diners rave about klongs inventive thai dishes and stylish ambiance the restaurants modern approach to traditional flavors and impeccable presentation earn it high praise among patrons with a

In [22]:
print(restaurants.columns)

Index(['Restaurant_Name', 'Restaurant_description', 'Location',
       'Restaurant_type', 'recommended_dishes', 'famous_Cuisines',
       'Cost_per_person', 'allergy_food_product', 'menu_without_allergy',
       'menu_with_allergy', 'reviews', 'Cleaned_Restaurant_Name',
       'Cleaned_Location', 'Cleaned_Restaurant_Type',
       'Cleaned_Recommended_Dishes', 'Cleaned_Famous_Cuisines',
       'Cleaned_Allergy_Food_Product', 'Cleaned_Menu_Without_Allergy',
       'Cleaned_Menu_With_Allergy', 'Cleaned_Reviews'],
      dtype='object')


In [23]:
print(restaurants.columns)

Index(['Restaurant_Name', 'Restaurant_description', 'Location',
       'Restaurant_type', 'recommended_dishes', 'famous_Cuisines',
       'Cost_per_person', 'allergy_food_product', 'menu_without_allergy',
       'menu_with_allergy', 'reviews', 'Cleaned_Restaurant_Name',
       'Cleaned_Location', 'Cleaned_Restaurant_Type',
       'Cleaned_Recommended_Dishes', 'Cleaned_Famous_Cuisines',
       'Cleaned_Allergy_Food_Product', 'Cleaned_Menu_Without_Allergy',
       'Cleaned_Menu_With_Allergy', 'Cleaned_Reviews'],
      dtype='object')


In [24]:
def longfamouscusinebased(Location, sentence, cuisine_sentence, Cost_per_person_sentence):
    restaurants['Cleaned_Location'] = restaurants['Cleaned_Location'].str.lower()
    restaurants['Cleaned_Restaurant_Type'] = restaurants['Cleaned_Restaurant_Type'].str.lower()
    restaurants['Cleaned_Famous_Cuisines'] = restaurants['Cleaned_Famous_Cuisines'].str.lower()
    restaurants['Cost_per_person'] = restaurants['Cost_per_person'].str.lower()

    sentence = sentence.lower()
    cuisine_sentence = cuisine_sentence.lower()
    Cost_per_person_sentence = Cost_per_person_sentence.lower()

    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()

    sentence_tokens = word_tokenize(sentence)
    cuisine_tokens = word_tokenize(cuisine_sentence)
    Cost_tokens = word_tokenize(Cost_per_person_sentence)

    sentence_tokens = [lemm.lemmatize(word) for word in sentence_tokens if word.isalnum() and word not in sw]
    cuisine_tokens = [lemm.lemmatize(word) for word in cuisine_tokens if word.isalnum() and word not in sw]
    Cost_tokens = [lemm.lemmatize(word) for word in Cost_tokens if word.isalnum() and word not in sw]

    reqbased = restaurants[restaurants['Cleaned_Location'] == Location.lower()].copy()
    similarities = []

    for index, row in reqbased.iterrows():
        # Tokenize the restaurant type and famous cuisines
        temp_tokens = word_tokenize(row['Cleaned_Restaurant_Type'])
        temp_tokens = [lemm.lemmatize(word) for word in temp_tokens if word.isalnum() and word not in sw]
        cuisine_tokens_copy = cuisine_tokens.copy()
        Cost_tokens_copy = Cost_tokens.copy()

        # Check for common tokens in restaurant type
        rvector = set(temp_tokens).intersection(set(sentence_tokens))

        # Check for common tokens in famous cuisines
        for cuisine_token in cuisine_tokens_copy:
            if cuisine_token in row['Cleaned_Famous_Cuisines']:
                rvector.add(cuisine_token)

        # Check for common tokens in Cost_per_person
        for Cost_token in Cost_tokens_copy:
            if Cost_token in row['Cost_per_person']:
                rvector.add(Cost_token)

        similarities.append(len(rvector))

    reqbased['similarity'] = similarities
    reqbased = reqbased.sort_values(by='similarity', ascending=False)

    return reqbased[['reviews', 'Restaurant_Name', 'Restaurant_description','Restaurant_type','recommended_dishes','menu_with_allergy','menu_without_allergy','allergy_food_product',
                     'Cost_per_person','reviews','similarity']].head(10)


    # Define the parameters
    location = 'New York City East Village Manhattan'
sentence = 'i want to eat from a Casual Dining restaurant'
cuisine_sentence = 'american'
Cost_per_person_sentence = '24'

# Call the function and store the result
result_df = longfamouscusinebased(location, sentence, cuisine_sentence, Cost_per_person_sentence)

# Print the parameters passed to the function
for index, row in result_df.iterrows():
    print("\nRestaurant Name:", row['Restaurant_Name'])
    print("Restaurant Description:", row['Restaurant_description'])
    print("Restaurant Type:", row['Restaurant_type'])
    print("Recommended Dishes:", row['recommended_dishes'])
    print("Menu with allergies:", row['menu_with_allergy'])
    print("Menu without allergies:", row['menu_without_allergy'])
    print("Allergy food product:", row['allergy_food_product'])
    print("Cost per person:", row['Cost_per_person'])
    print("Reviews:", row['reviews'])
    print("Similarity", row['similarity'])
    print("---------------------------------------------")



Restaurant Name: klong
Restaurant Description: Klong offers a modern interpretation of Thai cuisine, showcasing bold flavors and innovative dishes. From classic staples like pad Thai and green curry to creative offerings such as crispy duck salad and short rib curry, each dish at Klong is crafted with fresh, seasonal ingredients. Expect a harmonious blend of traditional Thai flavors with contemporary twists, resulting in a dining experience that is both exciting and satisfying.
Restaurant Type:  Casual Dining
Recommended Dishes: Pad Thai,Green Curry,Mango Sticky Rice,Tom Yum Soup,Massaman Curry,Greek Gyro Wrap
Menu with allergies: Greek Gyro Wrap
Menu without allergies: Pad Thai,Green Curry,Mango Sticky Rice,Tom Yum Soup,Massaman Curry
Allergy food product: Dairy
Cost per person: $29.05 
Reviews: reviews    diners rave about klongs inventive thai dishes...
reviews    diners rave about klongs inventive thai dishes...
Name: 326, dtype: object
Similarity 2
-------------------------------

In [25]:
#part 2
#if user enter long answers
#Location must be required
#location
def locationbased(Location):
    restaurants['Cleaned_Location']= restaurants['Cleaned_Location'].str.lower()
    locationbase=restaurants[restaurants['Cleaned_Location']==Location.lower()]
    if(locationbase.empty==0):
        hname=locationbase[['reviews', 'Restaurant_Name', 'Restaurant_description','Restaurant_type','recommended_dishes','menu_with_allergy','menu_without_allergy','allergy_food_product',
                     'Cost_per_person','reviews']]
        return hname.head()
    else:
        print('No restaurants Available')

#Define the parameters
location = 'Midtown Manhattan New York City'


# Call the function and store the result
result_df = locationbased(location)

# Print the parameters passed to the function
for index, row in result_df.iterrows():
    print("\nRestaurant Name:", row['Restaurant_Name'])
    print("Restaurant Description:", row['Restaurant_description'])
    print("Restaurant Type:", row['Restaurant_type'])
    print("Recommended Dishes:", row['recommended_dishes'])
    print("Menu with allergies:", row['menu_with_allergy'])
    print("Menu without allergies:", row['menu_without_allergy'])
    print("Allergy food product:", row['allergy_food_product'])
    print("Cost per person:", row['Cost_per_person'])
    print("Reviews:", row['reviews'])
    print("---------------------------------------------")





Restaurant Name: club a steakhouse
Restaurant Description: Club A Steakhouse is a premier steakhouse located in [Location], offering an upscale dining experience for steak enthusiasts. With its luxurious decor and elegant ambiance, Club A Steakhouse provides patrons with a refined setting to enjoy the finest cuts of USDA Prime beef, expertly prepared and cooked to perfection. In addition to its stellar steak offerings, the restaurant also features an extensive wine list, handcrafted cocktails, and an array of delectable appetizers and side dishes.
Restaurant Type: Fine Dining
Recommended Dishes: Filet Mignon,Porterhouse Steak,Lobster Bisque,Creamed Spinach,New York Cheesecake,Lemon Poppy Seed Muffins
Menu with allergies: Lemon Poppy Seed Muffins
Menu without allergies: Filet Mignon,Porterhouse Steak,Lobster Bisque,Creamed Spinach,New York Cheesecake
Allergy food product: Wheat, Dairy
Cost per person: $13.00 
Reviews: reviews    my seafood cocktail had wonderful large lump c...
reviews

In [26]:
def longfamouscusinebased(Location, sentence, cuisine_sentence, allergy_food_sentence):
    restaurants['Cleaned_Location'] = restaurants['Cleaned_Location'].str.lower()
    restaurants['Cleaned_Restaurant_Type'] = restaurants['Cleaned_Restaurant_Type'].str.lower()
    restaurants['Cleaned_Famous_Cuisines'] = restaurants['Cleaned_Famous_Cuisines'].str.lower()
    restaurants['allergy_food_product'] = restaurants['allergy_food_product'].str.lower()

    sentence = sentence.lower()
    cuisine_sentence = cuisine_sentence.lower()
    allergy_food_sentence = allergy_food_sentence.lower()

    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()

    sentence_tokens = word_tokenize(sentence)
    cuisine_tokens = word_tokenize(cuisine_sentence)
    allergy_tokens = word_tokenize(allergy_food_sentence)

    sentence_tokens = [lemm.lemmatize(word) for word in sentence_tokens if word.isalnum() and word not in sw]
    cuisine_tokens = [lemm.lemmatize(word) for word in cuisine_tokens if word.isalnum() and word not in sw]
    allergy_tokens = [lemm.lemmatize(word) for word in allergy_tokens if word.isalnum() and word not in sw]

    reqbased = restaurants[restaurants['Cleaned_Location'] == Location.lower()].copy()
    similarities = []
    for index, row in reqbased.iterrows():
        # Tokenize the restaurant type and famous cuisines
        temp_tokens = word_tokenize(row['Cleaned_Restaurant_Type'])
        temp_tokens = [lemm.lemmatize(word) for word in temp_tokens if word.isalnum() and word not in sw]
        cuisine_tokens_copy = cuisine_tokens.copy()
        allergy_tokens_copy = allergy_tokens.copy()

        # Check for common tokens in restaurant type
        rvector = set(temp_tokens).intersection(set(sentence_tokens))

        # Check for common tokens in famous cuisines
        for cuisine_token in cuisine_tokens_copy:
            if cuisine_token in row['Cleaned_Famous_Cuisines']:
                rvector.add(cuisine_token)

         # Check for common tokens in allergy food products
        for allergy_token in allergy_tokens_copy:
            if allergy_token in row['allergy_food_product']:
                rvector.add(allergy_token)

        similarities.append(len(rvector))


    reqbased['similarity'] = similarities
    reqbased = reqbased.sort_values(by='similarity', ascending=False)

    return reqbased[['reviews', 'Restaurant_Name', 'Restaurant_description','Restaurant_type','recommended_dishes','menu_with_allergy','menu_without_allergy','allergy_food_product',
                     'Cost_per_person','reviews','similarity']].head(10)


# Define parameters
location = 'New York City East Village Manhattan'
sentence = 'i want to eat from a Casual Dining restaurant'
cuisine =  'american'
allergy_food = 'im allergic to Dairy'

# Call the function and store the result
result_df = longfamouscusinebased(location, sentence, cuisine, allergy_food)

# Print the parameters passed to the function
for index, row in result_df.iterrows():
    print("\nRestaurant Name:", row['Restaurant_Name'])
    print("Restaurant Description:", row['Restaurant_description'])
    print("Restaurant Type:", row['Restaurant_type'])
    print("Recommended Dishes:", row['recommended_dishes'])
    print("Menu with allergies:", row['menu_with_allergy'])
    print("Menu without allergies:", row['menu_without_allergy'])
    print("Allergy food product:", row['allergy_food_product'])
    print("Cost per person:", row['Cost_per_person'])
    print("Reviews:", row['reviews'])
    print("Similarity", row['similarity'])
    print("---------------------------------------------")


Restaurant Name: klong
Restaurant Description: Klong offers a modern interpretation of Thai cuisine, showcasing bold flavors and innovative dishes. From classic staples like pad Thai and green curry to creative offerings such as crispy duck salad and short rib curry, each dish at Klong is crafted with fresh, seasonal ingredients. Expect a harmonious blend of traditional Thai flavors with contemporary twists, resulting in a dining experience that is both exciting and satisfying.
Restaurant Type:  Casual Dining
Recommended Dishes: Pad Thai,Green Curry,Mango Sticky Rice,Tom Yum Soup,Massaman Curry,Greek Gyro Wrap
Menu with allergies: Greek Gyro Wrap
Menu without allergies: Pad Thai,Green Curry,Mango Sticky Rice,Tom Yum Soup,Massaman Curry
Allergy food product: dairy
Cost per person: $29.05 
Reviews: reviews    diners rave about klongs inventive thai dishes...
reviews    diners rave about klongs inventive thai dishes...
Name: 326, dtype: object
Similarity 3
-------------------------------

In [27]:
#Location must be required
#location and restaurant type

def longrestauranttypebased(Location, sentence):
    restaurants['Cleaned_Location'] = restaurants['Cleaned_Location'].str.lower()
    restaurants['Cleaned_Restaurant_Type'] = restaurants['Cleaned_Restaurant_Type'].str.lower()

    sentence = sentence.lower()
    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()
    sentence_tokens = word_tokenize(sentence)
    sentence_tokens = [lemm.lemmatize(word) for word in sentence_tokens if word.isalnum() and word not in sw]

    reqbased = restaurants[restaurants['Cleaned_Location'] == Location.lower()].copy()
    similarities = []
    for index, row in reqbased.iterrows():
        temp_tokens = word_tokenize(row['Cleaned_Restaurant_Type'])
        temp_tokens = [lemm.lemmatize(word) for word in temp_tokens if word.isalnum() and word not in sw]
        rvector = set(temp_tokens).intersection(set(sentence_tokens))
        similarities.append(len(rvector))
    reqbased['similarity'] = similarities
    reqbased = reqbased.sort_values(by='similarity', ascending=False)
    return reqbased[['reviews', 'Restaurant_Name', 'Restaurant_description','Restaurant_type','recommended_dishes','menu_with_allergy','menu_without_allergy','allergy_food_product',
                     'Cost_per_person','reviews','similarity']].head(10)


# Define parameters
location = 'New York City East Village Manhattan'
sentence = 'i want to eat from a Casual Dining restaurant'

# Call the function and store the result
result_df = longrestauranttypebased(location, sentence)

# Print the parameters passed to the function
for index, row in result_df.iterrows():
    print("\nRestaurant Name:", row['Restaurant_Name'])
    print("Restaurant Description:", row['Restaurant_description'])
    print("Restaurant Type:", row['Restaurant_type'])
    print("Recommended Dishes:", row['recommended_dishes'])
    print("Menu with allergies:", row['menu_with_allergy'])
    print("Menu without allergies:", row['menu_without_allergy'])
    print("Allergy food product:", row['allergy_food_product'])
    print("Cost per person:", row['Cost_per_person'])
    print("Reviews:", row['reviews'])
    print("Similarity", row['similarity'])
    print("---------------------------------------------")


Restaurant Name: klong
Restaurant Description: Klong offers a modern interpretation of Thai cuisine, showcasing bold flavors and innovative dishes. From classic staples like pad Thai and green curry to creative offerings such as crispy duck salad and short rib curry, each dish at Klong is crafted with fresh, seasonal ingredients. Expect a harmonious blend of traditional Thai flavors with contemporary twists, resulting in a dining experience that is both exciting and satisfying.
Restaurant Type:  Casual Dining
Recommended Dishes: Pad Thai,Green Curry,Mango Sticky Rice,Tom Yum Soup,Massaman Curry,Greek Gyro Wrap
Menu with allergies: Greek Gyro Wrap
Menu without allergies: Pad Thai,Green Curry,Mango Sticky Rice,Tom Yum Soup,Massaman Curry
Allergy food product: dairy
Cost per person: $29.05 
Reviews: reviews    diners rave about klongs inventive thai dishes...
reviews    diners rave about klongs inventive thai dishes...
Name: 326, dtype: object
Similarity 2
-------------------------------

In [28]:
####################################Location must be required
#location ,restaurant type,famouse cuisines,allergy food product and cost_per_person

def longfamouscusinebased(Location, sentence, cuisine_sentence,allergy_food_sentence,Cost_per_person_sentence):
    restaurants['Cleaned_Location'] = restaurants['Cleaned_Location'].str.lower()
    restaurants['Cleaned_Restaurant_Type'] = restaurants['Cleaned_Restaurant_Type'].str.lower()
    restaurants['Cleaned_Famous_Cuisines'] = restaurants['Cleaned_Famous_Cuisines'].str.lower()
    restaurants['allergy_food_product'] = restaurants['allergy_food_product'].str.lower()
    restaurants['Cost_per_person'] = restaurants['Cost_per_person'].str.lower()


    sentence = sentence.lower()
    cuisine_sentence = cuisine_sentence.lower()
    allergy_food_sentence = allergy_food_sentence.lower()
    Cost_per_person_sentence = Cost_per_person_sentence.lower()

    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()

    sentence_tokens = word_tokenize(sentence)
    cuisine_tokens = word_tokenize(cuisine_sentence)
    allergy_tokens = word_tokenize(allergy_food_sentence)
    Cost_tokens = word_tokenize(Cost_per_person_sentence)

    sentence_tokens = [lemm.lemmatize(word) for word in sentence_tokens if word.isalnum() and word not in sw]
    cuisine_tokens = [lemm.lemmatize(word) for word in cuisine_tokens if word.isalnum() and word not in sw]
    allergy_tokens = [lemm.lemmatize(word) for word in allergy_tokens if word.isalnum() and word not in sw]
    Cost_tokens =[lemm.lemmatize(word) for word in Cost_tokens if word.isalnum() and word not in sw]

    reqbased = restaurants[restaurants['Cleaned_Location'] == Location.lower()].copy()
    similarities = []
    for index, row in reqbased.iterrows():
        # Tokenize the restaurant type and famous cuisines
        temp_tokens = word_tokenize(row['Cleaned_Restaurant_Type'])
        temp_tokens = [lemm.lemmatize(word) for word in temp_tokens if word.isalnum() and word not in sw]
        cuisine_tokens_copy = cuisine_tokens.copy()
        allergy_tokens_copy = allergy_tokens.copy()
        Cost_tokens_copy =Cost_tokens.copy()

        # Check for common tokens in restaurant type
        rvector = set(temp_tokens).intersection(set(sentence_tokens))

        # Check for common tokens in famous cuisines
        for cuisine_token in cuisine_tokens_copy:
            if cuisine_token in row['Cleaned_Famous_Cuisines']:
                rvector.add(cuisine_token)

         # Check for common tokens in allergy food products
        for allergy_token in allergy_tokens_copy:
            if allergy_token in row['allergy_food_product']:
                rvector.add(allergy_token)

        # Check for common tokens in allergy food products
        for Cost_token in Cost_tokens_copy:
            if Cost_token in row['Cost_per_person']:
                rvector.add(Cost_token)



        similarities.append(len(rvector))


    reqbased['similarity'] = similarities
    reqbased = reqbased.sort_values(by='similarity', ascending=False)

    return reqbased[['reviews', 'Restaurant_Name', 'Restaurant_description','Restaurant_type','recommended_dishes','menu_with_allergy','menu_without_allergy','allergy_food_product',
                     'Cost_per_person','reviews','similarity']].head(10)

    # Define the parameters
location = 'Colombo Sri Lanka'
sentence = 'i want to eat from a Fine Dining restaurant'
cuisine = 'i like Sri Lankan'
allergy = 'im alergic to pineapple'
cost = '3000'

# Call the function and store the result
result_df = longfamouscusinebased(location, sentence, cuisine, allergy, cost)

# Print the result
for index, row in result_df.iterrows():
    print("\nRestaurant Name:", row['Restaurant_Name'])
    print("Restaurant Description:", row['Restaurant_description'])
    print("Restaurant Type:", row['Restaurant_type'])
    print("Recommended Dishes:", row['recommended_dishes'])
    print("Menu with allergies:", row['menu_with_allergy'])
    print("Menu without allergies:", row['menu_without_allergy'])
    print("Allergy food product:", row['allergy_food_product'])
    print("Cost per person:", row['Cost_per_person'])
    print("Reviews:", row['reviews'])
    print("Similarity", row['similarity'])
    print("---------------------------------------------")


Restaurant Name: spice island
Restaurant Description: Spice Island is an exotic dining destination nestled in Colombo, Sri Lanka, renowned for its diverse menu and authentic Sri Lankan flavors. With a rating of 4.9, it offers a culinary journey through the island's rich heritage. Approximate cost for two is 8000 Sri Lankan Rupees. Contact +9411-8765432 for reservations.
Restaurant Type: Fine Dining, Culinary Adventure
Recommended Dishes: String Hoppers, Fish Ambul Thiyal,Crab Curry,String Hoppers,Watalappam,Seafood Risotto,Blueberry Cheesecake,Chicken Kottu,Fish Fries with cheese,Garlic Prawns
Menu with allergies: String Hoppers, Fish Ambul Thiyal,Crab Curry,String Hoppers,Watalappam,Seafood Risotto,Blueberry Cheesecake,Chicken Kottu,Fish Fries with cheese,Garlic Prawns
Menu without allergies: Fish Ambul Thiyal,Crab Curry,Fish Fries with cheese,Garlic Prawns,,Blueberry Cheesecake
Allergy food product: gluten
Cost per person: 8000 sri lankan rupees
Reviews: reviews    spice island deli

In [29]:
#part 3
#if user adked long based questions

import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

def longfamouscusinebased(sentence):
    restaurants['merged_info'] = restaurants[['Location', 'Restaurant_type', 'famous_Cuisines', 'allergy_food_product', 'Cost_per_person', 'recommended_dishes']].fillna('').agg(', '.join, axis=1)
    restaurants['merged_info'] = restaurants['merged_info'].str.lower()

    sentence = sentence.lower()

    sw = stopwords.words('english')
    lemm = WordNetLemmatizer()

    sentence_tokens = word_tokenize(sentence)
    sentence_tokens = [lemm.lemmatize(word) for word in sentence_tokens if word.isalnum() and word not in sw]

    reqbased = restaurants.copy()
    similarities = []
    for index, row in reqbased.iterrows():
        # Tokenize the merged information
        temp_tokens = word_tokenize(row['merged_info'])
        temp_tokens = [lemm.lemmatize(word) for word in temp_tokens if word.isalnum() and word not in sw]

        # Check for common tokens
        rvector = set(temp_tokens).intersection(set(sentence_tokens))
        similarities.append(len(rvector))

    reqbased['similarity'] = similarities
    reqbased = reqbased.sort_values(by='similarity', ascending=False)

    return reqbased[['reviews', 'Restaurant_Name', 'Restaurant_description','Restaurant_type','recommended_dishes','menu_with_allergy','menu_without_allergy','allergy_food_product',
                     'Cost_per_person','similarity']].head(10)

    # Define the parameters
sentence = 'i want to eat from a Casual Dining restaurant from sri lanka'

# Call the function and store the result
result_df = longfamouscusinebased(sentence)

# Print the parameters passed to the function
for index, row in result_df.iterrows():
    print("\nRestaurant Name:", row['Restaurant_Name'])
    print("Restaurant Description:", row['Restaurant_description'])
    print("Restaurant Type:", row['Restaurant_type'])
    print("Recommended Dishes:", row['recommended_dishes'])
    print("Menu with allergies:", row['menu_with_allergy'])
    print("Menu without allergies:", row['menu_without_allergy'])
    print("Allergy food product:", row['allergy_food_product'])
    print("Cost per person:", row['Cost_per_person'])
    print("Reviews:", row['reviews'])
    print("Similarity", row['similarity'])
    print("---------------------------------------------")


Restaurant Name: curry pot
Restaurant Description: Curry Pot offers an authentic Sri Lankan dining experience in the heart of Colombo, Sri Lanka, with a rating of 4.6. Known for its rich curries and flavorful rice dishes.
Restaurant Type: Casual Dining, Authentic
Recommended Dishes: Fish Ambul Thiyal, Polos Curry, String Hoppers, Sri Lankan Beef Curry
Menu with allergies: Fish Ambul Thiyal, Polos Curry, String Hoppers, Sri Lankan Beef Curry
Menu without allergies: Fish Ambul Thiyal, Polos Curry,  Sri Lankan Beef Curry
Allergy food product: gluten, dairy
Cost per person: 2800 lkr
Reviews: curry pot provides a delightful taste of sri lankan cuisine with its rich curries and flavorful rice dishes earning it high praise
Similarity 4
---------------------------------------------

Restaurant Name: sri lankan spice house
Restaurant Description: The Sri Lankan Spice House is a cozy eatery situated in Colombo, Sri Lanka, with a commendable rating of 4.8. Renowned for its authentic Sri Lankan d

In [30]:
import pandas as pd
from nltk.tokenize import word_tokenize

# Read the CSV file containing bad words
badwords_df = pd.read_csv('nowords.csv')

# Assuming the column name containing bad words is 'badwords'
badwords = set(badwords_df['splitword'].str.lower())

def check_for_bad_words(sentence):
    # Tokenize the user's sentence and lowercase the words
    sentence_tokens = word_tokenize(sentence.lower())

    # Check if any of the words from the user's sentence match the bad words
    for word in sentence_tokens:
        if word in badwords:
            return True  # Bad word found

    return False  # No bad words found

# Example usage:
user_input = "I'm in New York City book."
if check_for_bad_words(user_input):
    print("I'm sorry, I'm not able to assist with that. Could you please ask me something related to restaurants?")

I'm sorry, I'm not able to assist with that. Could you please ask me something related to restaurants?


In [31]:
import pickle

In [32]:
file_path = '/content/model_data.pkl'

In [42]:
import h5py
import numpy as np

In [40]:
file_path = '/content/model1_data.h5'